In [1]:
from app.dotenv import base_dir, data_dir
import os
from sentence_transformers import SentenceTransformer

pdf_path = f"{data_dir}/{os.getenv('pdf_file')}.pdf"
print('pdf_path:',pdf_path )


embedding_model_name = os.getenv('embedding_model_name')
print('embeding model:',embedding_model_name )
embedding_model = SentenceTransformer(embedding_model_name)  # You can choose any available model

llm_model_name = os.getenv('llm_model_name')
print('llm model:',llm_model_name )


pdf_path: /Users/yishai/Desktop/InsuranceHebrewRAG-Assistant/data/כתב שירות רפואה משלימה.pdf
embeding model: imvladikon/sentence-transformers-alephbert
llm model: aminadaven/dictalm2.0-instruct:f16


load the hebrew pdf data and format it

In [2]:
from app.pdf_reader import read_heb_pdf
full_text = read_heb_pdf(pdf_path)
print('full_text: \n',full_text[:500])

full_text: 
 תמצית תנאי הביטוח - כתב שירות - רפואה משלימה (3603)
תמצית פרטי הפוליסה
שם הביטוח כתב שירות - רפואה משלימה
סוג הביטוח כתב שירות
תקופת הביטוח מתחדשת כל שנתיים.ראה סעיף 10 לכתב השירות
תיאור הביטוח שירותי רפואה משלימה ופיזיותרפיה
הפוליסה אינה מכסה את המבוטח
ראה סעיף 6 לכתב השירות
במקרים הבאים (חריגים בפוליסה)
אחרי כמה זמן מתחילת הביטוח
90 ימים.ראה סעיף 17.2 לכתב השירות
ניתן לתבוע ולקבל תגמול 1)הרשכא(
קבלת השירות מנותן שירות שבהסכם -
סך של 47 ₪ בעבור כל טיפול ו/או התייעצות שיבוצעו במרפאת נותן השירות 


split text into chunks (using RecursiveCharacterTextSplitter)

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Create an instance of RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)

# Split the text
chunks = text_splitter.split_text(full_text)
# print(f'data is split into {len(chunks)} chunks' )

# Print the resulting chunks
for chunk in chunks [200:300]:
    print('_'*50)
    print(chunk)

__________________________________________________
ערב השינוי,לפי הגבוה מבניהם,או הפחתה בהיקף השירותים שנעשתה כתחליף להעלאת דמי המנוי כאמור (להלן:
__________________________________________________
"שינוי הדורש אישור") תידרש הסכמתו המפורשת של המנוי להמשך הביטוח,ויחולו ההוראות הבאות:
__________________________________________________
1.2.10 שינוי הדורש אישור יבחן בנפרד ביחס לכתב שירות זה.למרות האמור,במקרה של שיווק בחבילה,יבחן
__________________________________________________
השינוי גם ביחס לחבילה כולה.
__________________________________________________
2.2.10 לא התקבלה הסכמתו המפורשת של המנוי לשינוי הדורש אישור - יסתיים כתב השירות במועד השינוי
__________________________________________________
וזאת למעט ביחס למנוי שהגיש במהלך תקופת השירות הנוכחית או זו שקדמה לה,תביעה למימוש זכויות
__________________________________________________
לפי תנאי כתב השירות."הסכמתו המפורשת של המנוי" - לגבי ילד עד גיל 21 של מנוי באותו כתב שירות
__________________________________________________
או בן זוגו של מ

create rag (since its a single document DB is in-memory)

In [4]:
from app.rag import *
rag_index = create_RAG(chunks, embedding_model)


get answer to user query (ree text):
1. retrieve relevant data from rag
2. create a prompt with question and context and chain of thoughts
3. recieve answer from the llm model (ollama api)
4. display user query and llm response

In [5]:
from app.llm import load_and_format_prompt,prompt_llm,print_prompt_and_response



# Example usage
user_query = "האם מקבלים החזר מלא על הטיפולים"

retrieved_chunks = retrieve_context(user_query, chunks, embedding_model, rag_index, print_results=False)
# Format the final prompt
prompt = load_and_format_prompt(user_query, retrieved_chunks)
llm_response = prompt_llm(llm_model_name,prompt)
print_prompt_and_response(user_query, llm_response)


===== User Input Query =====
האם מקבלים החזר מלא על הטיפולים

===== Final LLM Response =====

The refund policy for the treatments covered by your insurance plan is 50% of the actual payment made for each treatment, up to a maximum of 140 NIS per treatment. However, this applies only to treatments that have already been received and not to future or promised treatments. To qualify for the refund, you must be an active member of the insurance policy, and the reimbursement will be limited to costs actually incurred, not any payments made beyond those expenses. This benefit is subject to certain limitations and exclusions as specified in your plan's contract.


In [10]:

from app.metrics import bertSimilarity


# Encode sentences in Hebrew
sentence1,sentences2 = "שלום, איך אתה?", "מה שלומך?"
sentence1,sentences2 ="כן אתה תקבל החזר", ""
print("Cosine Similarity:", bertSimilarity(sentence1,sentences2))



Cosine Similarity: 0.9907829


In [18]:


import json
import time

# Function to load questions from a JSON file
def load_questions_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

# Function to load and display questions one by one
def load_questions(data):
    for item in data:
        question = item["question"]
        answer = item["answer"]
        topic = item["topic"]
        
        # Print the question, answer, and topic
        print(f"Question: {question}")
        print(f"Answer: {answer}")
        print(f"Topic: {topic}")
        print("-" * 50)
        
        # Add a small delay before loading the next question (to simulate loading one by one)
        time.sleep(2)

# Main function to run the script
def main():
    file_path = 'app/metrics/questions.json'  # Path to your JSON file
    data = load_questions_from_file(file_path)
    
    load_questions(data)


main()



Question: מהי תקופת הביטוח?
Answer: תקופת הביטוח מתחדשת כל שנתיים.
Topic: תקופת הביטוח
--------------------------------------------------
Question: האם ניתן לתבוע תגמול לאחר 90 ימים מתחילת הביטוח?
Answer: כן, ניתן לתבוע ולבקש תגמול לאחר 90 ימים מיום תחילת הביטוח.
Topic: תביעות
--------------------------------------------------
Question: כמה זמן יש לחכות לפני שניתן לתבוע פיצויים?
Answer: יש לחכות 90 ימים מתחילת הביטוח כדי להתחיל בתהליך תביעה.
Topic: תביעות
--------------------------------------------------
Question: מה קורה אם לא קיבלתי את השירות במרפאת נותן השירות שבהסכם?
Answer: במידה ולא קיבלת את השירות במרפאת נותן השירות שבהסכם, תוכל לקבל שיפוי בגין ההוצאות בפועל, אך לא יותר מ-140 ₪ לכל טיפול.
Topic: שירותים
--------------------------------------------------
Question: מהי השתתפות העצמית בביטוח?
Answer: השתתפות עצמית היא סכום שהמנוי חייב לשלם בעבור כל טיפול או ייעוץ, בכפוף לכיסוי שהוצג בכתב השירות.
Topic: השתתפות עצמית
--------------------------------------------------
Question: מהם 

KeyboardInterrupt: 